In [22]:
!pip install osmnx
!pip install utm

You should consider upgrading via the 'C:\Users\HP\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.
C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\utils\_process_win32.py:145: ResourceWarning: unclosed file <_io.BufferedWriter name=4>
  return process_handler(cmd, _system_body)
C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\utils\_process_win32.py:145: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  return process_handler(cmd, _system_body)
C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\utils\_process_win32.py:145: ResourceWarning: unclosed file <_io.BufferedReader name=6>
  return process_handler(cmd, _syst

You should consider upgrading via the 'C:\Users\HP\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [23]:
import pandas as pd
import xgboost as xgb
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, average_precision_score, roc_auc_score, classification_report
import osmnx
import utm
import numpy as np

In [24]:
df = pd.read_csv("final_ev.csv")
df = df.drop(['Unnamed: 0'], axis=1)

In [25]:
df.head()

,latitude,longitude,college,food_court,hospital,parking,restaurant,university,charging_station
0,51.510359,-0.187704,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,51.403444,-0.255276,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,51.465290,-0.058371,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,51.438312,-0.156274,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,51.358167,-0.216632,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [26]:
X = df.drop(['charging_station'], axis=1)
y=df['charging_station']

In [27]:
X = X.to_numpy()
y = y.to_numpy()

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=0, test_size=0.2)

In [29]:
cls = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
cls.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=42,
              reg_alpha=0, reg_lambda=1, ...)

In [30]:
labels = cls.predict(X_train)
print(classification_report(y_train, labels))
print('Confusion Matrix')
print(confusion_matrix(y_train, labels))

              precision    recall  f1-score   support

         0.0       0.92      1.00      0.96      3492
         1.0       1.00      0.90      0.95      2960

    accuracy                           0.95      6452
   macro avg       0.96      0.95      0.95      6452
weighted avg       0.96      0.95      0.95      6452

Confusion Matrix
[[3489    3]
 [ 291 2669]]


In [31]:
labels = cls.predict(X_test)
print(classification_report(y_test, labels))
print('Confusion Matrix')
print(confusion_matrix(y_test, labels))

              precision    recall  f1-score   support

         0.0       0.89      0.98      0.93       914
         1.0       0.97      0.83      0.90       700

    accuracy                           0.92      1614
   macro avg       0.93      0.91      0.92      1614
weighted avg       0.92      0.92      0.92      1614

Confusion Matrix
[[899  15]
 [116 584]]


In [32]:
pickle.dump(cls, open("optimal_pred_model3.pkl", "wb"))

C:\Users\HP\AppData\Local\Temp/ipykernel_11728/529154887.py:1: ResourceWarning: unclosed file <_io.BufferedWriter name='optimal_pred_model3.pkl'>
  pickle.dump(cls, open("optimal_pred_model3.pkl", "wb"))


In [33]:
import math
def round_down(num,digits):
  factor = 10.0 ** digits
  return math.floor(num * factor) / factor


In [34]:
def predict_optimal_place(lat,lon):
  new_model = pickle.load(open("optimal_pred_model3.pkl", "rb"))

  # converting lat, lon from WGS4 to UT,
  easting, northing, _, _ = utm.from_latlon(lat, lon)

  # adding 250 metres buffer to the user input coordinates to give surrounding amenity data
  easting = easting + 250
  northing = northing + 250

  # converting back to lat, lon

  lat1, lon1 = utm.to_latlon(easting, northing, 30, 'U')
  
  # amenities that we are considering
  tags = {'amenity': ['food_court', 'restaurant', 'college', 'university', 'parking', 'hospital']}
  
  # fetching suitable amenity location through OpenStreetMap
  x = osmnx.geometries.geometries_from_bbox(north=lat1, south=lat, east=lon1, west=lon, tags=tags)

  x = x[['amenity']]
  x = pd.get_dummies(x['amenity'])

  possible_amenities = {
      'food_court' : 0,
      'restaurant' : 0,
      'college' : 0,
      'university': 0,
      'parking': 0, 
      'hospital': 0
 }

  captured_variables = x.columns.values.tolist()

  for amenity in captured_variables:
    if x[amenity].sum()>0:
      possible_amenities.update({amenity:1})
  
  ip_features = np.array([lat, lon, possible_amenities['college'], possible_amenities['food_court'], possible_amenities['hospital'], possible_amenities['parking'], possible_amenities['restaurant'], possible_amenities['university']])
  
  pred = new_model.predict(ip_features.reshape(-1,1).T)

  del new_model

  return pred


In [21]:
x

NameError: name 'x' is not defined

In [35]:
predict_optimal_place(6.6,48.6)

C:\Users\HP\AppData\Local\Temp/ipykernel_11728/1377367806.py:2: ResourceWarning: unclosed file <_io.BufferedReader name='optimal_pred_model3.pkl'>
  new_model = pickle.load(open("optimal_pred_model3.pkl", "rb"))


array([0])